In [ ]:
import pandas as pd
import numpy as np
import folium
import json
import matplotlib.pyplot as pp
%matplotlib inline

from IPython.display import display, HTML

In [ ]:
ur_nat_raw = pd.read_excel("amstat_nat.xlsx")
ur_all_raw = pd.read_excel("amstat_all.xlsx")

cant_to_id = {
    "Berne": "BE",
    "Soleure": "SO",
    "Argovie": "AG",
    "Bâle-Campagne": "BL",
    "Bâle-Ville": "BS",
    "Appenzell Rhodes-Extérieures": "AR",
    "Appenzell Rhodes-Intérieures": "AI",
    "Glaris": "GL",
    "Grisons": "GR",
    "Schaffhouse": "SH",
    "St-Gall": "SG",
    "Thurgovie": "TG",
    "Lucerne": "LU",
    "Nidwald": "NW",
    "Obwald":"OW",
    "Schwyz": "SZ",
    "Uri": "UR",
    "Zoug": "ZG",
    "Zurich": "ZH",
    "Fribourg":"FR",
    "Jura":"JU",
    "Neuchâtel": "NE",
    "Genève": "GE",
    "Valais": "VS",
    "Vaud": "VD",
    "Tessin": "TI"
}

ur_nat_raw["cant_id"] = ur_nat_raw["cant"].map(cant_to_id)
ur_all_raw["cant_id"] = ur_all_raw["cant"].map(cant_to_id)

ur_nat = ur_nat_raw.set_index(["ling_reg", "reg", "cant", "cant_id", "nat"])
ur = ur_all_raw.set_index(["ling_reg", "reg", "cant", "cant_id"])

for col in ["unempl_c", "jobseek_c", "empl_jobseek_c"]:
    ur_nat[col] = pd.to_numeric(ur_nat[col].str.replace("'", ""))

ur["rate_nat_diff"] = ur_nat.xs("Etrangers", level="nat")["rate"] - ur_nat.xs("Suisses", level="nat")["rate"]
ur

In [ ]:
cantons_b = json.load(open("topojson/ch-cantons.topojson.json"))

In [ ]:
m = folium.Map([46.90, 8.50], zoom_start=8)

m.choropleth(geo_data=cantons_b,
             threshold_scale=[0., 1.5, 3.8, 4.5, 5],                   # Compute automaticall ? see next cell
             topojson="objects.cantons", 
             data=ur["rate"].reset_index(level=[0,1,2], drop=True), 
             fill_color='OrRd', 
             key_on="id",
             legend_name="Unemployment Rate")
m

In [ ]:
pp.plot(ur["rate"], np.zeros_like(ur["rate"]), 'x')
pp.show()                                           # Shall we automate this or is it good enough ?

In [ ]:
# Second map of pt. 2 : the rate is already computed this way... what shall we do ?

In [ ]:
m = folium.Map([46.90, 8.50], zoom_start=8)
pp.plot(ur["rate_nat_diff"], np.zeros_like(ur["rate_nat_diff"]), 'x')
pp.show()        

m.choropleth(geo_data=cantons_b,
             threshold_scale=[0., 1.1, 2.2, 3.2, 4,5.5],                   # Compute automaticall ? see next cell
             topojson="objects.cantons", 
             data=ur["rate_nat_diff"].reset_index(level=[0,1,2], drop=True), 
             fill_color='OrRd', 
             key_on="id",
             legend_name="Unemployment Rates difference")
m